\begin{center} 
Chun-Yuan (Scott) Chiu 
\end{center}
\begin{center} 
chunyuac@andrew.cmu.edu 
\end{center}

# 1. {-}

## (a) {-}

In [1]:
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans

# data = pd.read_csv('http://rosmarus.refsmmat.com/datasets/datasets/nutrition/sr28_ABBREV.csv', index_col=0)

fulldata = pd.read_csv('sr28_ABBREV.csv', index_col=0).dropna()
data = fulldata[['Energ_Kcal', 'Protein', 'Lipid_Tot', 'Carbohydrt', 'Fiber_TD', 'Sugar_Tot', 'Cholestrl']]

fulldata['Cluster'] = KMeans(n_clusters=5).fit_predict(scale(data))
fulldata[['Shrt_Desc', 'Cluster']][3:39]

,Shrt_Desc,Cluster
NDB_No,,
1004,"CHEESE,BLUE",0
1005,"CHEESE,BRICK",0
1006,"CHEESE,BRIE",0
1007,"CHEESE,CAMEMBERT",0
1009,"CHEESE,CHEDDAR",0
1011,"CHEESE,COLBY",0
1012,"CHEESE,COTTAGE,CRMD,LRG OR SML CURD",1
1013,"CHEESE,COTTAGE,CRMD,W/FRUIT",1
1014,"CHEESE,COTTAGE,NONFAT,UNCRMD,DRY,LRG OR SML CURD",1


In [19]:
import hvplot.pandas
from pandas import DataFrame
import matplotlib.pyplot as plt

columns = data[3:39].columns
index = data[3:39].index

DataFrame(scale(data[3:39]), columns=columns, index=index).hvplot()   #.hvplot()
# plt.show()

:NdOverlay   [Variable]
   :Curve   [NDB_No]   (value)

## (b) {-}